In [1]:
import torch
import torch.nn as nn
import torch.functional as F


In [2]:
torch.__version__

'2.5.1+cu118'

In [3]:
import torch
import sys

print(f"Python版本: {sys.version}")
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA是否可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA版本: {torch.version.cuda}")
    print(f"当前GPU设备: {torch.cuda.current_device()}")
    print(f"GPU设备名称: {torch.cuda.get_device_name()}")
    print(f"可用GPU数量: {torch.cuda.device_count()}")
else:
    print("CUDA不可用 - 可能是CPU版本的PyTorch")

Python版本: 3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)]
PyTorch版本: 2.5.1+cu118
CUDA是否可用: True
CUDA版本: 11.8
当前GPU设备: 0
GPU设备名称: NVIDIA GeForce RTX 4060 Laptop GPU
可用GPU数量: 1
